In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import trading_vix_and_spy_utils

In [2]:
data_names = []
data_names.append("./hourly_data/VIX_1050D_1hour_TRADES.csv")
data_names.append("./hourly_data/VIX_1050D_1hour_OPTION_IMPLIED_VOLATILITY.csv")
data_names.append("./hourly_data/VIXY_1050D_1hour_BID.csv")
data_names.append("./hourly_data/VIXY_1050D_1hour_ASK.csv")
data_names.append("./hourly_data/VIXY_1050D_1hour_OPTION_IMPLIED_VOLATILITY.csv")
data_names.append("./hourly_data/SPY_1050D_1hour_BID.csv")
data_names.append("./hourly_data/SPY_1050D_1hour_ASK.csv")
data_names.append("./hourly_data/SPY_1050D_1hour_OPTION_IMPLIED_VOLATILITY.csv")
new_column_names = []
new_column_names.append('vix_index')
new_column_names.append('vix_index_iv')
new_column_names.append('vixy_bid')
new_column_names.append('vixy_ask')
new_column_names.append('vixy_iv')
new_column_names.append('spy_bid')
new_column_names.append('spy_ask')
new_column_names.append('spy_iv')

In [3]:
for index in range(0,len(data_names)):
    
    data_name = data_names[index]
    new_column_name = new_column_names[index]
    
    if index == 0:
        
        total_data = pd.read_csv(data_name)
        total_data = total_data.rename(columns = {"date":"date",\
                                "open":new_column_name+"_open",\
                                "high":new_column_name+'_high',\
                                'low':new_column_name+'_low',\
                                'close':new_column_name+'_close',\
                                'volume':new_column_name+'_volume'})
    
    else:
        
        new_data = pd.read_csv(data_name)
        new_data = new_data.rename(columns = {"date":"date",\
                                "open":new_column_name+"_open",\
                                "high":new_column_name+'_high',\
                                'low':new_column_name+'_low',\
                                'close':new_column_name+'_close',\
                                'volume':new_column_name+'_volume'})
        
        total_data = pd.merge(total_data,new_data,on = 'date',how = 'inner')

In [4]:
distiled_column_names = []
for entry in total_data.columns:
    if entry == "date":
        distiled_column_names.append(entry)
    elif 'close' in entry:
        distiled_column_names.append(entry)
        
total_data_distiled = pd.DataFrame()
for entry in distiled_column_names:
    total_data_distiled[entry] = total_data[entry]

In [5]:
total_data_distiled.head()

,date,vix_index_close,vix_index_iv_close,vixy_bid_close,vixy_ask_close,vixy_iv_close,spy_bid_close,spy_ask_close,spy_iv_close
0,20180827 09:30:00,12.25,0.906911,93.88,93.92,0.647569,289.30,289.31,0.087707
1,20180827 10:00:00,12.21,0.940390,94.36,94.40,0.649553,289.47,289.48,0.087707
2,20180827 11:00:00,12.23,0.869637,94.40,94.44,0.643680,289.82,289.83,0.087262
3,20180827 12:00:00,12.15,0.868669,94.08,94.16,0.634504,289.83,289.84,0.087564
4,20180827 13:00:00,12.21,0.874654,94.24,94.28,0.637679,289.72,289.73,0.087707


In [6]:
total_data_distiled.tail()

,date,vix_index_close,vix_index_iv_close,vixy_bid_close,vixy_ask_close,vixy_iv_close,spy_bid_close,spy_ask_close,spy_iv_close
4954,20210702 11:00:00,14.55,0.999332,23.06,23.07,0.752674,432.44,432.45,0.102899
4955,20210702 12:00:00,14.65,0.990665,23.16,23.17,0.752404,433.26,433.27,0.103010
4956,20210702 13:00:00,14.82,0.996983,23.40,23.41,0.772517,433.31,433.32,0.103200
4957,20210702 14:00:00,14.92,1.001602,23.35,23.36,0.770279,433.26,433.27,0.104200
4958,20210702 15:00:00,14.98,1.000284,23.62,23.67,0.729735,433.75,433.76,0.104677


In [7]:
total_data_distiled.shape

(4959, 9)

In [8]:
def compute_slope_feature(input_column_name,data_window):
    
    linear_regression_length = data_window
    linear_regressor = LinearRegression() 
    x = [i for i in range(0,linear_regression_length)]
    x = np.asarray(x)
    x = np.reshape(x,(-1,1))

    slope_data = []

    for end_index in range(total_data_distiled.shape[0],linear_regression_length-1,-1):
        y = total_data_distiled[input_column_name].iloc[end_index-linear_regression_length:end_index].tolist()
        y = np.asarray(y)
        y = np.reshape(y,(-1,1))

        linear_regressor.fit(x,y)

        slope_data.append(linear_regressor.coef_[0][0])

    #add some zeros before
    for _ in range(0,linear_regression_length-1):
        slope_data.append(0)
        
    return slope_data[::-1]

In [9]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_iv_close',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [10]:
index_begin_at = total_data_distiled.columns.tolist().index('vix_index_close')+1

In [11]:

window_lengths = [10,30,50,100,200]

for index in range(0,len(window_lengths)):
    window_length = window_lengths[index]
    new_col_index = index_begin_at+index #vix_index_close originally has column index 1
    new_col_name = 'vix_index_'+str(window_length)+"_slope"
    
    new_data = compute_slope_feature('vix_index_close',window_length)
    total_data_distiled.insert(loc = new_col_index,column = new_col_name,value = new_data)


In [12]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [13]:
total_data_distiled.head()

,date,vix_index_close,vix_index_10_slope,vix_index_30_slope,vix_index_50_slope,vix_index_100_slope,vix_index_200_slope,vix_index_iv_close,vixy_bid_close,vixy_ask_close,vixy_iv_close,spy_bid_close,spy_ask_close,spy_iv_close
0,20180827 09:30:00,12.25,0.0,0.0,0.0,0.0,0.0,0.906911,93.88,93.92,0.647569,289.30,289.31,0.087707
1,20180827 10:00:00,12.21,0.0,0.0,0.0,0.0,0.0,0.940390,94.36,94.40,0.649553,289.47,289.48,0.087707
2,20180827 11:00:00,12.23,0.0,0.0,0.0,0.0,0.0,0.869637,94.40,94.44,0.643680,289.82,289.83,0.087262
3,20180827 12:00:00,12.15,0.0,0.0,0.0,0.0,0.0,0.868669,94.08,94.16,0.634504,289.83,289.84,0.087564
4,20180827 13:00:00,12.21,0.0,0.0,0.0,0.0,0.0,0.874654,94.24,94.28,0.637679,289.72,289.73,0.087707


In [14]:
total_data_distiled.tail()

,date,vix_index_close,vix_index_10_slope,vix_index_30_slope,vix_index_50_slope,vix_index_100_slope,vix_index_200_slope,vix_index_iv_close,vixy_bid_close,vixy_ask_close,vixy_iv_close,spy_bid_close,spy_ask_close,spy_iv_close
4954,20210702 11:00:00,14.55,-0.139636,-0.042031,-0.013891,-0.027053,-0.012845,0.999332,23.06,23.07,0.752674,432.44,432.45,0.102899
4955,20210702 12:00:00,14.65,-0.135576,-0.048211,-0.014809,-0.028419,-0.012657,0.990665,23.16,23.17,0.752404,433.26,433.27,0.103010
4956,20210702 13:00:00,14.82,-0.121030,-0.051657,-0.015032,-0.029504,-0.012550,0.996983,23.40,23.41,0.772517,433.31,433.32,0.103200
4957,20210702 14:00:00,14.92,-0.091394,-0.052618,-0.015521,-0.030314,-0.012372,1.001602,23.35,23.36,0.770279,433.26,433.27,0.104200
4958,20210702 15:00:00,14.98,-0.059939,-0.052968,-0.015964,-0.031087,-0.012169,1.000284,23.62,23.67,0.729735,433.75,433.76,0.104677


In [15]:
#build features for vix based on vix6 trading environment
#compute the exponential moving average
mv_10 = total_data_distiled['vix_index_close'].ewm(span = 10).mean()
mv_20 = total_data_distiled['vix_index_close'].ewm(span = 20).mean()
mv_30 = total_data_distiled['vix_index_close'].ewm(span = 30).mean()
mv_50 = total_data_distiled['vix_index_close'].ewm(span = 50).mean()
mv_100 = total_data_distiled['vix_index_close'].ewm(span = 100).mean()

spot_to_mv_10 = total_data_distiled['vix_index_close']/mv_10
spot_to_mv_20 = total_data_distiled['vix_index_close']/mv_20
spot_to_mv_30 = total_data_distiled['vix_index_close']/mv_30
spot_to_mv_50 = total_data_distiled['vix_index_close']/mv_50
spot_to_mv_100 = total_data_distiled['vix_index_close']/mv_100

vix_measure = spot_to_mv_10+spot_to_mv_20+spot_to_mv_30+spot_to_mv_50+spot_to_mv_100
vix_measure_list = vix_measure.tolist()

index_begin_at = total_data_distiled.columns.tolist().index('vix_index_iv_close')+1
total_data_distiled.insert(loc = index_begin_at,column = 'vix_index_mv_ratio',value = vix_measure_list)

index_begin_at = total_data_distiled.columns.tolist().index('vix_index_mv_ratio')+1
threshold_list = [5.5,6,6.5]
col_index_adder = 0
for threshold in threshold_list:
    counting_days = trading_vix_and_spy_utils.day_counter_helper(vix_measure_list,threshold)
    total_data_distiled.insert(loc = index_begin_at+col_index_adder,column = 'vix_index_intervals_since_'+str(threshold),value = counting_days)
    col_index_adder += 1

In [16]:
total_data_distiled.columns.tolist()

['date',
 'vix_index_close',
 'vix_index_10_slope',
 'vix_index_30_slope',
 'vix_index_50_slope',
 'vix_index_100_slope',
 'vix_index_200_slope',
 'vix_index_iv_close',
 'vix_index_mv_ratio',
 'vix_index_intervals_since_5.5',
 'vix_index_intervals_since_6',
 'vix_index_intervals_since_6.5',
 'vixy_bid_close',
 'vixy_ask_close',
 'vixy_iv_close',
 'spy_bid_close',
 'spy_ask_close',
 'spy_iv_close']

In [ ]:
vixy_mid_close = (total_data_distiled['vixy_bid_close']+total_data_distiled['vixy_ask_close'])/2
total_data_distiled.insert(loc = 14,column = 'vixy_mid_close',value = vixy_mid_close)

In [ ]:
total_data_distiled.columns

In [ ]:
total_data_distiled

In [ ]:
window_lengths = [10,30,50,100,200]

for index in range(0,len(window_lengths)):
    window_length = window_lengths[index]
    new_col_index = 12+index #vix_index_close originally has column index 1
    new_col_name = 'vixy_mid_close_'+str(window_length)+"_slope"
    
    new_data = compute_slope_feature('vixy_mid_close',window_length)
    total_data_distiled.insert(loc = new_col_index,column = new_col_name,value = new_data)


In [ ]:
total_data_distiled.columns.tolist()

In [ ]:
total_data_distiled.columns.tolist().index('date')

In [ ]:
total_data_distiled

In [ ]:
spy_mid_close = (total_data_distiled['spy_bid_close']+total_data_distiled['spy_ask_close'])/2
total_data_distiled.insert(loc = 24,column = 'spy_mid_close',value = spy_mid_close)

In [ ]:
total_data_distiled

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(total_data_distiled['vix_index_close'])
plt.plot(total_data_distiled['vix_index_30_slope'])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vix_index_30_slope'])
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(total_data_distiled['vix_index_30_slope'])
result

In [ ]:
plt.plot(total_data_distiled['vix_index_30_slope'][-300:])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vix_index_iv_close'][-300:])
plt.show()

In [ ]:
result = adfuller(total_data_distiled['vix_index_iv_close'])
result

In [ ]:
result = adfuller(total_data_distiled['vix_index_close'])
result

In [ ]:
plt.plot(total_data_distiled['vix_index_intervals_since_5.5'])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vix_index_intervals_since_6'])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vix_index_intervals_since_6.5'])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vix_index_mv_ratio'])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vixy_mid_close_50_slope'])
plt.show()

In [ ]:
plt.plot(total_data_distiled['vix_index_50_slope'])
plt.show()